## 🍃 Carbon Calculation Chaos
The basic formula for calculating embodied carbon is:

$$
\text{Embodied Carbon} = \sum \left(\text{Material Quantity} \times \text{Emission Factor}\right)
$$

Where:
- **Material Quantity** is the amount of each construction material.
- **Emission Factor** is the carbon intensity of each material (e.g., kgCO₂e per unit of material).

For the **Aquatics Centre** project, we are tasked at providing a high-level computation of the embodied carbon based on the above formula.

Equation variables:
- **Material Quantity** is obtained from the geometric Revit model. Volume (m³)(from Revit) multiplied by the density (kg/m³)(known) returns mass (kg).
- **Emission Factor** is provided in tabular form (from Excel) in kgCO₂e/kg.

#### 🏃 Step 01 - Receiving commit in a condensed fashion

In [ ]:
# Import required libraries
from specklepy.api.client import SpeckleClient
from specklepy.core.api import operations
from specklepy.transports.server import ServerTransport

# Define global variables 
HOST = "https://app.speckle.systems/"
AUTHENTICATION_TOKEN = ""
STREAM_ID = ""
BRANCH_NAME = ""

# Setting up SpeckleClient and authenticating
client = SpeckleClient(host=HOST)
client.authenticate_with_token(token=AUTHENTICATION_TOKEN)

# Receving commit
transport = ServerTransport(STREAM_ID, client)
branch = client.branch.get(stream_id=STREAM_ID, name=BRANCH_NAME)
model_data = operations.receive(branch.commits.items[0].referencedObject, transport)

#### 🔍 Step 02 - Where To Start?
Maybe the question is, where on earth do I even start??
* `model_data` is at the root of our commit, below visualized as the `structural` object
* `elements` represents the Revit objects grouped by their types (Walls, Columns etc.)
* We want to build a dataframe of all object types relevant for our basic calculation
* A safe function to retrieve the attribute is Python's `gettattr()` function, which allows us to do some sort of "error-handling" 

![alt text](<Screenshot 2024-11-14 093304.png>)

In [ ]:
# Using the above explanation, we can very easily obtain our model objects
elements = getattr(model_data, "elements", None)

if elements:
    print("Success! We have elements, now we can proceed with filtering.")
    assert len(elements) == 10 # The size of the array (from the screenshot) indicates, that their should be 10 different groups. NOTE: It is usually not a good idea to hard-code these numbers as we have variability from version to version 

####  ⏳ Step 03 - Filter our RVT data
So, we have an array of **ALL** objects. But, we want to just have the elements relevant to our calculation. How do we go about this?
* Each `element` within `elements` has a `name` property.
* What names (i.e. categories) do we have?

In [ ]:
print([element.name for element in elements])

For our calculation, we are just looking at the basic structural objects. Using some basic list comprehension, we can filter our `elements` list for just the information of interest to us.

In [ ]:
# Some basic filtering
wanted_elements = ["Structural Foundation", "Walls", "Structural Columns", "Floors"]
filtered_elements = # TODO

####  🐼 Step 04 - Thinking about our Pandas Dataframe / Data Structure
- A well-organised dataframe makes post-processing, computations, groupings and other insights easy to obtain.
- The information may be structured as follows:

In [ ]:
import pandas as pd
from IPython.display import display

# List of headers
headers = ["id", "category", "level", "material", "volume", "density", "mass", "emission_factor", "embodied_carbon"]

# Create an empty DataFrame with the specified headers
df = pd.DataFrame(columns=headers)
display(df)

#### 🔩 Step 05 - Reinforcing some Structural Concepts
- Reinforced concrete elements are comprised of two materials: concrete and steel
- Both materials have separate emission factors and thus need to be separated for our computations
- Thankfully, the Excel "model" provides us with reinforcment rates as a way to estimate the quantity of reinforcing steel in a concrete element
- Concrete volume (m³) x Reinforcement rate (kg/m³) = Steel mass (kg)

In [ ]:
# Receiving the Excel data
branch = client.branch.get(stream_id=STREAM_ID, name="database")
model_data = operations.receive(branch.commits.items[0].referencedObject, transport)
emission_factor_data = model_data.data
emission_factor_data

#### 🔨 Step 06 - Building our Pandas Dataframe
Looping through (primitive, I know) the elements and poppulating the dataframe accordingly. Thankfully, our range of materials is limited making conditional statements simple. For concrete elements, we need an extra step to split between concrete and steel.

The "loop logic" looks as follows:
- Determine material type
- Using the material and object type (or group), supplement information with the density and emission factor from the Excel data
- If concrete, compute mass of steel add row data for both materials.

In [ ]:
# TODO

In [ ]:
# Calculate the total embodied carbon
total_embodied_carbon = df['embodied_carbon'].sum()

# Display the result
print(f"Total Embodied Carbon: {total_embodied_carbon:.2f} kg CO2")

#### 🤩 Step 07 - Visualize

Imports and Setup

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Set up Seaborn and Matplotlib styles for modern, appealing visuals
sns.set_theme(style="whitegrid")
plt.style.use("ggplot")


Pie Chart Embodied Carbon by Material

In [ ]:
# Calculate total embodied carbon per material
carbon_by_material = df.groupby("material")["embodied_carbon"].sum()

# Plot pie chart
fig = px.pie(
    names=carbon_by_material.index,
    values=carbon_by_material.values,
    title="Embodied Carbon Distribution by Material",
    color_discrete_sequence=px.colors.sequential.RdBu
)
fig.update_traces(textinfo="percent+label", pull=[0.1 if i == max(carbon_by_material.values) else 0 for i in carbon_by_material.values])
fig.show()


Pie Chart: Embodied Carbon by Level

In [ ]:
# Calculate total embodied carbon per level
carbon_by_level = df.groupby("level")["embodied_carbon"].sum()

# Plot pie chart
fig = px.pie(
    names=carbon_by_level.index,
    values=carbon_by_level.values,
    title="Embodied Carbon Distribution by Level",
    color_discrete_sequence=px.colors.sequential.Teal
)
fig.update_traces(textinfo="percent+label", pull=[0.1 if i == max(carbon_by_level.values) else 0 for i in carbon_by_level.values])
fig.show()

Bar Chart: Embodied Carbon by Category

In [ ]:
# Calculate total embodied carbon per category
carbon_by_category = df.groupby("category")["embodied_carbon"].sum().sort_values()

# Plot bar chart with updated 'hue' handling and font fixes
plt.figure(figsize=(10, 6))
sns.barplot(x=carbon_by_category, y=carbon_by_category.index, hue=carbon_by_category.index, palette="viridis")
plt.title("Embodied Carbon by Category")
plt.xlabel("Total Embodied Carbon (kg CO₂e)")
plt.ylabel("Category")

# Fix missing glyph warning by changing font to one that supports subscript characters
plt.rcParams["font.family"] = "DejaVu Sans"  # A font that supports subscript/superscript

plt.show()

Stacked Bar Chart: Embodied Carbon per Material for Each Level

In [ ]:
# Calculate total embodied carbon per material per level
carbon_material_level = df.groupby(["level", "material"])["embodied_carbon"].sum().unstack()

# Plot stacked bar chart
carbon_material_level.plot(kind="bar", stacked=True, colormap="coolwarm", figsize=(10, 6))
plt.title("Embodied Carbon per Material Across Levels")
plt.xlabel("Level")
plt.ylabel("Embodied Carbon (kg CO₂e)")
plt.legend(title="Material", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.show()